In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from fastai.text.all import *
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/reddit-headlines-updated/headlines.xlsx


In [2]:
df=pd.read_excel("/kaggle/input/reddit-headlines-updated/headlines.xlsx", header=0, names=['Headline','Sarcasm'])
df.head()

,Headline,Sarcasm
0,"PornHub Surprises Frequent User With Wife, Loving Family Upon 10,000th Masturbation",1
1,"Physical Therapy Office Politely Declines Daniel Jones’ Offer Of Framed, Signed Jersey For Wall",1
2,Delight At Receiving Breakfast In Bed Mitigated By Difficulty Of Eating While Horizontal,1
3,Tips For Getting Diagnosed With ADHD As An Adult,1
4,Fact-Checking J.D. Vance’s ‘Hillbilly Elegy’,1


In [3]:
dls_lm=DataBlock(blocks=(TextBlock.from_df('Headline', is_lm=True)),get_x=ColReader('text'), splitter=RandomSplitter(0.2))

In [4]:
dls_lm = dls_lm.dataloaders(df, bs=32)

dls_lm.show_batch(max_n=1)

,text,text_
0,"xxbos xxmaj xxunk accused of xxunk a xxmaj nazi xxunk during the xxmaj xxunk xxunk gets xxunk 5 years in prison xxbos xxmaj apple xxmaj unveils xxmaj new xxmaj apple xxmaj vision xxmaj pro xxmaj xxunk xxbos xxmaj xxunk xxmaj xxunk xxmaj to xxmaj xxunk xxmaj xxunk xxmaj of xxmaj sexual xxmaj xxunk , xxmaj drug xxmaj use xxmaj against xxmaj matt xxmaj xxunk xxbos xxmaj xxunk xxmaj new xxmaj report xxmaj","xxmaj xxunk accused of xxunk a xxmaj nazi xxunk during the xxmaj xxunk xxunk gets xxunk 5 years in prison xxbos xxmaj apple xxmaj unveils xxmaj new xxmaj apple xxmaj vision xxmaj pro xxmaj xxunk xxbos xxmaj xxunk xxmaj xxunk xxmaj to xxmaj xxunk xxmaj xxunk xxmaj of xxmaj sexual xxmaj xxunk , xxmaj drug xxmaj use xxmaj against xxmaj matt xxmaj xxunk xxbos xxmaj xxunk xxmaj new xxmaj report xxmaj finds"


In [6]:
lm_learn=language_model_learner(dls_lm, AWD_LSTM, metrics=[accuracy,Perplexity()]).to_fp16()
lm_learn.fit_one_cycle(1,2e-2)
lm_learn.unfreeze()
lm_learn.fit_one_cycle(15, 2e-3)
lm_learn.save_encoder('finetuned')

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.567890,3.059771,0.568680,21.322670,00:02


epoch,train_loss,valid_loss,accuracy,perplexity,time
0,2.988567,2.751930,0.512835,15.672844,00:01
1,2.791917,2.551102,0.581742,12.821224,00:01
2,2.652452,2.437198,0.586661,11.440935,00:01
3,2.537686,2.372732,0.594990,10.726654,00:01
4,2.420918,2.381957,0.595403,10.826072,00:01
5,2.308074,2.363503,0.605138,10.628119,00:01
6,2.198707,2.399745,0.600550,11.020369,00:01
7,2.136125,2.613751,0.582982,13.650159,00:01
8,2.058110,2.483351,0.582651,11.981353,00:01
9,1.970163,2.495064,0.576637,12.122504,00:01


In [12]:
text="Trump"
n_w=10
n_s=1
pred=[lm_learn.predict(text, n_w, temperature=0.5) for _ in range(n_s)]
print("\n".join(pred))

Trump Boys Help Father Raise $ 50


In [13]:
blocks = (TextBlock.from_df('Headline', seq_len=dls_lm.seq_len, vocab=dls_lm.vocab), CategoryBlock())
dls = DataBlock(blocks=blocks,
                get_x=ColReader('text'),
                get_y=ColReader('Sarcasm'),
                splitter=RandomSplitter(0.2))
dls = dls.dataloaders(df, bs=32)


In [14]:
learn_clas = text_classifier_learner(dls, AWD_LSTM, metrics=[accuracy, FBeta(beta=1)]).to_fp16()
learn_clas.load_encoder('finetuned')

In [16]:
learn_clas.fit_one_cycle(1,2e-2)
learn_clas.freeze_to(-2)
learn_clas.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))
learn_clas.freeze_to(-3)
learn_clas.fit_one_cycle(1,slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,fbeta_score,time
0,0.361218,0.152414,0.944700,0.969388,00:01


epoch,train_loss,valid_loss,accuracy,fbeta_score,time
0,0.298587,0.097634,0.990783,0.994737,00:01


epoch,train_loss,valid_loss,accuracy,fbeta_score,time
0,0.219297,0.075849,0.995392,0.997375,00:01


In [17]:
learn_clas.unfreeze()
learn_clas.fit_one_cycle(5,slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,fbeta_score,time
0,0.133889,0.073926,0.995392,0.997375,00:02
1,0.137943,0.072051,0.995392,0.997375,00:01
2,0.118166,0.054821,0.995392,0.997375,00:01
3,0.104297,0.054024,0.995392,0.997375,00:01
4,0.091037,0.046691,0.995392,0.997375,00:01


In [21]:
learn_clas.predict("Delhi flooding LIVE: Police writes to MCD seeking de-silting of drains")

('0', tensor(0), tensor([0.8533, 0.1467]))

In [23]:
learn_clas.predict("Completely Alone Man Really Thought Blowing Out Birthday Candle In Dark Apartment Would Have Cheered Him Up")

('1', tensor(1), tensor([0.0167, 0.9833]))

In [22]:
learn_clas.export("model.pkl")